<img src="http://vision.skills.network/logo-light.png" width="400" alt="CV Studio logo"  />


<h2>Transfer Learning with Convolutional Neural Networks For Classification with PyTorch and   <a href="https://vision.skills.network"> Computer Vision Learning
Studio
 (CV Studio)</a></h2> <p><b> V 0.2</b></p>
<h4>Project: Final_project_stop_signs</h4>
<h4>Training Run: Task 2 Train or classify</h4>


Estimated time needed: **40** minutes


In this lab, you will train a deep neural network for  image classification using <a href="https://cs231n.github.io/transfer-learning/">transfer learning</a>, the image dataset will automatically be download from your <a href="https://vision.skills.network/">CV Studio</a> account. Experiment with different hyperparameters.


## Objectives


In this lab you will train a state of the art image classifier using and <a href="https://vision.skills.network/">CV Studio</a>, CV Studio is a fast, easy and collaborative open source image annotation tool for teams and individuals. In practice, very few people train an entire Convolutional Network from scratch (with random initialization), because it is relatively rare to have a dataset of sufficient size. Instead, it is common to pretrain a ConvNet on a very large dataset in the lab, then use this Network to train your model. We will use the Convolutional Network as a feature generator, only training the output layer.  In general, 100-200 images will give you a good starting point, and it only takes about half an hour.  Usually, the more images you add, the better your results, but it takes longer and the rate of improvement will decrease.


-   Import Libraries and Define Auxiliary Functions
-   Create Dataset Object
-   Load Model and Train


* * *


## Import Libraries and Define Auxiliary Functions


In [1]:
# ! conda install -c pytorch torchvision
# ! pip install skillsnetwork tqdm
# !pip install  skillsnetwork

Libraries for OS and Cloud


In [2]:
import os
import uuid
import shutil
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import copy
from datetime import datetime

Libraries for Data Processing and Visualization


In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from matplotlib.pyplot import imshow
from tqdm import tqdm
from ipywidgets import IntProgress
import time
import zipfile

Deep Learning Libraries


In [4]:
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim import lr_scheduler
from torchvision import transforms
import torch.nn as nn
torch.manual_seed(0)

In [5]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2024-05-16 00:08:57--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.62.142.15, 2600:1407:3c00:f9b::317f, 2600:1407:3c00:f9d::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.62.142.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M   109MB/s    in 7.4s    

2024-05-16 00:09:05 (106 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [6]:
source_path = '/tmp/PetImages'

source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')

# Deletes all non-image files (there are two .db files bundled into the dataset)
!find /tmp/PetImages/ -type f ! -name "*.jpg" -exec rm {} +

# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12500 images of dogs.
There are 12500 images of cats.


In [7]:
root_dir = '/tmp/cats-v-dogs'

if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

Plot train cost and validation accuracy:


In [8]:
def plot_stuff(COST,ACC):
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color = color)
    ax1.set_xlabel('Iteration', color = color)
    ax1.set_ylabel('total loss', color = color)
    ax1.tick_params(axis = 'y', color = color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color = color)  # we already handled the x-label with ax1
    ax2.plot(ACC, color = color)
    ax2.tick_params(axis = 'y', color = color)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    plt.show()

Plot the transformed image:


In [9]:
def imshow_(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.permute(1, 2, 0).numpy()
    print(inp.shape)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
    plt.show()

Compare the prediction and actual value:


In [10]:
def result(model,x,y):
    #x,y=sample
    z=model(x.unsqueeze_(0))
    _,yhat=torch.max(z.data, 1)

    if yhat.item()!=y:
        text="predicted: {} actual: {}".format(str(yhat.item()),y)
        print(text)

Define our device as the first visible cuda device if we have CUDA available:


In [11]:
def create_train_val_dirs(root_path):
  """
  Creates directories for the train and test sets

  Args:
    root_path (string) - the base directory path to create subdirectories from

  Returns:
    None
  """
  training = os.path.join(root_path, "training")
  os.makedirs(training , exist_ok=True)

  training_dogs = os.path.join(training, "dogs")
  os.makedirs(training_dogs , exist_ok=True)

  training_cats = os.path.join(training, "cats")
  os.makedirs(training_cats , exist_ok=True)

  validation = os.path.join(root_path, "validation")
  os.makedirs(validation , exist_ok=True)

  validation_dogs = os.path.join(validation, "dogs")
  os.makedirs(validation_dogs , exist_ok=True)

  validation_cats = os.path.join(validation, "cats")
  os.makedirs(validation_cats , exist_ok=True)

In [12]:
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [13]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/tmp/cats-v-dogs/training
/tmp/cats-v-dogs/validation
/tmp/cats-v-dogs/training/cats
/tmp/cats-v-dogs/training/dogs
/tmp/cats-v-dogs/validation/cats
/tmp/cats-v-dogs/validation/dogs


In [14]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets

  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training

  Returns:
    None
  """
  os.makedirs(TRAINING_DIR, exist_ok=True)
  os.makedirs(VALIDATION_DIR, exist_ok=True)

  files = os.listdir(SOURCE_DIR)
  num_train = int(len(files) * SPLIT_SIZE)


  for i, file in enumerate(files):
        src_file = os.path.join(SOURCE_DIR, file)
        if os.path.getsize(src_file) > 0:
            if i < num_train:
                dest_file = os.path.join(TRAINING_DIR, file)
            else:
                dest_file = os.path.join(VALIDATION_DIR, file)
            shutil.copyfile(src_file, dest_file)
        else:
          print(f"{file} is zero length, so ignoring.")

In [15]:
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"

TRAINING_DIR = "/tmp/cats-v-dogs/training/"
VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"\n\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(VALIDATION_CATS_DIR))} images of cats for validation")
print(f"There are {len(os.listdir(VALIDATION_DOGS_DIR))} images of dogs for validation")

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


Original cat's directory has 12500 images
Original dog's directory has 12500 images

There are 11249 images of cats for training
There are 11249 images of dogs for training
There are 1250 images of cats for validation
There are 1250 images of dogs for validation


In [16]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators

  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images

  Returns:
    train_generator, validation_generator - tuple containing the generators
  """
  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale=1/255.0)

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode="binary",
                                                      target_size=(150, 150))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale=1/255.0)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode="binary",
                                                                target_size=(150, 150))
  return train_generator, validation_generator

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("the device type is", device)

the device type is cuda:0


## Load Data


In this section we will preprocess our dataset by changing the shape of the image, converting to tensor and normalizing the image channels. These are the default preprocessing steps for image data. In addition, we will perform data augmentation on the training dataset. The preprocessing steps for the test dataset is the same, but W do not prform data augmentation on the test dataset.


<code>
<p>mean = [0.485, 0.456, 0.406]</p>
<p>std = [0.229, 0.224, 0.225]</p>
<p>composed = transforms.Compose([transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),transforms.RandomRotation(degrees=5)
                               , transforms.ToTensor()
                               , transforms.Normalize(mean, std)])</p>
    </code>


Download the data:


We need to get our training and validation dataset. 90% of the data will be used for training.


In [ ]:
TRAINING_CATS_DIR

In [33]:
for file in os.scandir(VALIDATION_DOGS_DIR):
    print
     (file.path)

/tmp/cats-v-dogs/validation/dogs/2469.jpg
/tmp/cats-v-dogs/validation/dogs/8079.jpg
/tmp/cats-v-dogs/validation/dogs/11395.jpg
/tmp/cats-v-dogs/validation/dogs/2074.jpg
/tmp/cats-v-dogs/validation/dogs/4684.jpg
/tmp/cats-v-dogs/validation/dogs/272.jpg
/tmp/cats-v-dogs/validation/dogs/10543.jpg
/tmp/cats-v-dogs/validation/dogs/12304.jpg
/tmp/cats-v-dogs/validation/dogs/812.jpg
/tmp/cats-v-dogs/validation/dogs/6461.jpg
/tmp/cats-v-dogs/validation/dogs/6175.jpg
/tmp/cats-v-dogs/validation/dogs/3265.jpg
/tmp/cats-v-dogs/validation/dogs/914.jpg
/tmp/cats-v-dogs/validation/dogs/10743.jpg
/tmp/cats-v-dogs/validation/dogs/5210.jpg
/tmp/cats-v-dogs/validation/dogs/11673.jpg
/tmp/cats-v-dogs/validation/dogs/11525.jpg
/tmp/cats-v-dogs/validation/dogs/7632.jpg
/tmp/cats-v-dogs/validation/dogs/6405.jpg
/tmp/cats-v-dogs/validation/dogs/12381.jpg
/tmp/cats-v-dogs/validation/dogs/6374.jpg
/tmp/cats-v-dogs/validation/dogs/8103.jpg
/tmp/cats-v-dogs/validation/dogs/4761.jpg
/tmp/cats-v-dogs/validation/do

In [ ]:
def resize_generator(generator):
    processed_images = []

    # Iterar sobre os lotes de imagens do gerador
    for images, _ in generator:
        # Iterar sobre as imagens em cada lote
        for image in images:
            try:
                # Redimensionar e normalizar a imagem
                transform = transforms.Compose([
                    transforms.ToPILImage(),  # Converter de tensor para imagem PIL
                    transforms.Resize((150, 150)),
                    transforms.ToTensor(),  # Converte a imagem para um tensor
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalização
                ])
                image = transform(image)

                # Armazenar o tensor resultante
                processed_images.append(image)

            except Exception as e:
                print(f"Error processing image: {str(e)}")

    return processed_images

In [18]:
def resize_generator(generator):
    processed_images = []

    # Iterar sobre os lotes de imagens do gerador
    for images, _ in generator:
        # Iterar sobre as imagens em cada lote
        for image in images:
            try:
                # Redimensionar e normalizar a imagem
                transform = transforms.Compose([
                    transforms.ToPILImage(),  # Converter de tensor para imagem PIL
                    transforms.Resize((150, 150)),
                    transforms.ToTensor(),  # Converte a imagem para um tensor
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalização
                ])
                image = transform(image)

                # Armazenar o tensor resultante
                processed_images.append(image)

            except Exception as e:
                print(f"Error processing image: {str(e)}")

    return processed_images

In [ ]:
percentage_train=0.9

# Gerar conjuntos de treinamento e validação
train_generator, val_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

# Redimensionar imagens nos conjuntos de treinamento e validação
train_set = resize_generator(train_generator)
val_set = resize_generator(val_generator)

We can plot some of our dataset:


In [ ]:
i=0

for x, y  in val_set:
    imshow_(x,"y=: {}".format(str(y.item())))
    i+=1
    if i==3:
        break

## Hyperparameters


Experiment with different hyperparameters:


<b>Epoch</b> indicates the number of passes of the entire training dataset, here we will set the number of epochs to 10:


In [ ]:
n_epochs=10

<b>Batch size</b> is the number of training samples utilized in one iteration. If the batch size is equal to the total number of samples in the training set, then every epoch has one iteration. In Stochastic Gradient Descent, the batch size is set to one. A batch size of 32--512 data points seems like a good value, for more information check out the following <a href="https://arxiv.org/abs/1609.04836?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-cvstudio-2021-01-01">link</a>.


In [ ]:
batch_size=32

 <b>Learning rate</b> is used in the training of neural networks. Learning rate is a hyperparameter with a small positive value, often in the range between 0.0 and 1.0.


In [ ]:
lr=0.000001

<b>Momentum</b> is a term used in the gradient descent algorithm to improve training results:


In [ ]:
momentum=0.9

If you set to <code>lr_scheduler=True</code>  for every epoch use a learning rate scheduler changes the range of the learning rate from a maximum or minimum value. The learning rate usually decays over time.


In [ ]:
lr_scheduler=True
base_lr=0.001
max_lr=0.01

# Load Model and Train


This function will train the model


In [ ]:
def train_model(model, train_loader,validation_loader, criterion, optimizer, n_epochs,print_=True):
    loss_list = []
    accuracy_list = []
    correct = 0
    #global:val_set
    n_test = len(val_set)
    accuracy_best=0
    best_model_wts = copy.deepcopy(model.state_dict())

    # Loop through epochs
        # Loop through the data in loader
    print("The first epoch should take several minutes")
    for epoch in tqdm(range(n_epochs)):

        loss_sublist = []
        # Loop through the data in loader

        for x, y in train_loader:
            x, y=x.to(device), y.to(device)
            model.train()

            z = model(x)
            loss = criterion(z, y)
            loss_sublist.append(loss.data.item())
            loss.backward()
            optimizer.step()

            optimizer.zero_grad()
        print("epoch {} done".format(epoch) )

        scheduler.step()
        loss_list.append(np.mean(loss_sublist))
        correct = 0


        for x_test, y_test in validation_loader:
            x_test, y_test=x_test.to(device), y_test.to(device)
            model.eval()
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / n_test
        accuracy_list.append(accuracy)
        if accuracy>accuracy_best:
            accuracy_best=accuracy
            best_model_wts = copy.deepcopy(model.state_dict())


        if print_:
            print('learning rate',optimizer.param_groups[0]['lr'])
            print("The validaion  Cost for each epoch " + str(epoch + 1) + ": " + str(np.mean(loss_sublist)))
            print("The validation accuracy for epoch " + str(epoch + 1) + ": " + str(accuracy))
    model.load_state_dict(best_model_wts)
    return accuracy_list,loss_list, model

 Load the pre-trained model resnet18. Set the parameter pretrained to true.


In [ ]:
model = models.resnet18(pretrained=True)

We will only train the last layer of the network set the parameter <code>requires_grad</code> to <code>False</code>, the network is a fixed feature extractor.


In [ ]:
for param in model.parameters():
  param.requires_grad = False

Number of classes


In [ ]:
n_classes = len(train_set.class_indices)
n_classes

Replace the output layer model.fc of the neural network with a nn.Linear object, to classify <code>n_classes</code> different classes. For the parameters in_features  remember the last hidden layer has 512 neurons.


In [ ]:
# Type your code here
model.fc = nn.Linear(512, n_classes)

Set device type


In [ ]:
model.to(device)

Cross-entropy loss, or log loss, measures the performance of a classification model combines LogSoftmax in one object class. It is useful when training a classification problem with C classes.


In [ ]:
criterion = nn.CrossEntropyLoss()

Create a training loader and validation loader object.


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_set , batch_size=batch_size,shuffle=True)
validation_loader= torch.utils.data.DataLoader(dataset=val_set , batch_size=1)

Use the optim package to define an Optimizer that will update the weights of the model for us.


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

We use <a href='https://arxiv.org/pdf/1506.01186.pdf'>Cyclical Learning Rates</a>


In [ ]:
if lr_scheduler:
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01,step_size_up=5,mode="triangular2")

Now we are going to train model,for 500 images this take 25 minutes, depending on your dataset


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-Coursera/images/this_make_take_time.gif" alt="this may take some time">


In [ ]:
start_datetime = datetime.now()
start_time=time.time()

accuracy_list,loss_list, model=train_model(model,train_loader , validation_loader, criterion, optimizer, n_epochs=n_epochs)

end_datetime = datetime.now()
current_time = time.time()
elapsed_time = current_time - start_time
print("elapsed time", elapsed_time )

Now run the following to report back the results of the training run to CV Studio


In [ ]:


parameters = {
    'epochs': n_epochs,
    'learningRate': lr,
    'momentum':momentum,
    'percentage used training':percentage_train,
    "learningRatescheduler": {"lr_scheduler":lr_scheduler,"base_lr":base_lr, "max_lr" :max_lr}


}
result = cvstudioClient.report(started=start_datetime, completed=end_datetime, parameters=parameters, accuracy={ 'accuracy': accuracy_list, 'loss': loss_list })

if result.ok:
    print('Congratulations your results have been reported back to CV Studio!')

Save the model to model.pt


In [ ]:
# Save the model to model.pt
torch.save(model.state_dict(), 'model.pt')

# Save the model and report back to CV Studio
result = cvstudioClient.uploadModel('model.pt', {'numClasses': n_classes})

Plot train cost and validation accuracy,  you can improve results by getting more data.


In [ ]:
plot_stuff(loss_list,accuracy_list)

Load the model that performs best:  


In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, n_classes)
model.load_state_dict(torch.load( "model.pt"))
model.eval()

In [ ]:
imageNames = ['stop_1.jpeg','stop_2.jpeg','not_stop_1.jpeg']
for imageName in imageNames:
    image = Image.open(imageName)
    transform = composed = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    x = transform(image)
    z=model(x.unsqueeze_(0))
    _,yhat=torch.max(z.data, 1)
    # print(yhat)
    prediction = "Stop"
    if yhat == 1:
        prediction ="Not Stop"
    imshow_(transform(image),imageName+": Prediction = "+prediction)